In [278]:
import pandas as pd
import pyodbc as mssql

In [279]:
conn = mssql.connect("DRIVER={SQL SERVER};SERVER=IPSBD;DATABASE=LIQUIDACION;Trusted_Connection=Yes;")
auditorias = pd.read_sql("SELECT * FROM AUDIT A WHERE FECHA >= 44562 AND A.ORIGEN='D' AND A.TIPO='M'", conn)
campos = pd.read_sql("""SELECT c.* FROM AUDIT A INNER JOIN CAMPOS C ON C.AUDIT=A.AUDIT WHERE FECHA >= 44562 AND A.ORIGEN='D' 
                     AND A.TIPO='M' and campo IN ('BENEFICIO','BENEFICIO(ANT)')""", conn)

In [280]:
campos.set_index(inplace=True, keys=["Audit", "Campo"])


In [281]:
campos = campos.unstack("Campo")
campos.columns = ["BENEFICIO", "BENEFICIO(ANT)"]

In [282]:
campos["BENEFICIO"] = campos["BENEFICIO"].apply(lambda x: x.replace(",","."))
campos["BENEFICIO(ANT)"] = campos["BENEFICIO(ANT)"].apply(lambda x: x.replace(",","."))

In [283]:
campos["BENEFICIO"] = campos["BENEFICIO"].astype(float)
campos["BENEFICIO(ANT)"] = campos["BENEFICIO(ANT)"].astype(float)

In [284]:
auditorias=auditorias.merge(right=campos, on="Audit")

In [285]:
#auditorias[auditorias["BENEFICIO"] < (auditorias["BENEFICIO(ANT)"] + (auditorias["BENEFICIO(ANT)"] * 0.35))]
auditorias = auditorias[auditorias["BENEFICIO"] < auditorias["BENEFICIO(ANT)"]]